<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-tools/blob/main/notebooks/Eye_ai_Image_Cropping_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Prerequisites
!pip install git+https://github.com/fair-research/bdbag
!pip install git+https://github.com/informatics-isi-edu/deriva-py
!pip install 'git+https://github.com/informatics-isi-edu/eye-ai-tools' --upgrade --force
!pip install 'git+https://github.com/informatics-isi-edu/deriva-ml' --upgrade --force

import json
import os
from eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePosixPath



  Cloning https://github.com/fair-research/bdbag to /tmp/pip-req-build-q1jyluo2
  Running command git clone --filter=blob:none --quiet https://github.com/fair-research/bdbag /tmp/pip-req-build-q1jyluo2
  Resolved https://github.com/fair-research/bdbag to commit f8d81362ceaff7a0328dd89257655cfcb3f7d111
  Preparing metadata (setup.py) ... done
  Created wheel for bdbag: filename=bdbag-1.7.2-py2.py3-none-any.whl size=74828 sha256=c1251651087930353845a3bdc9cfce55dc758a63840e8f09acaa17a659e3289f
  Stored in directory: /tmp/pip-ephem-wheel-cache-0l7g6ox4/wheels/e6/37/a1/ed335483cf88440ed32b8f918cc20fa31b2ee2c81dd9504f7a
Successfully built bdbag
  Cloning https://github.com/informatics-isi-edu/deriva-py to /tmp/pip-req-build-82qqtthm
  Running command git clone --filter=blob:none --quiet https://github.com/informatics-isi-edu/deriva-py /tmp/pip-req-build-82qqtthm
  Resolved https://github.com/informatics-isi-edu/deriva-py to commit 3dda9d8bda15f00434263d3509ef33895fdbdfd5
  Preparing metadata

In [2]:
# @title login to DERIVA via Globus Auth

DEBUG_MODE = True # False #@param
catalog_id = "eye-ai" #@param
DEFAULT_SERVER = 'dev.eye-ai.org' if DEBUG_MODE else 'www.eye-ai.org'

!deriva-globus-auth-utils login --no-browser --host {DEFAULT_SERVER}


2024-02-01 17:25:47,527 - INFO - No default configuration file found, attempting to create one at: /root/.deriva/config.json
2024-02-01 17:25:48,172 - INFO - NoSavedTokens: No tokens were loaded
Please paste the following URL in a browser:
https://auth.globus.org/v2/oauth2/authorize?client_id=8ef15ba9-2b4a-469c-a163-7fd910c9d111&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2Fdev.eye-ai.org%2Fderiva_all+openid+urn%3Aglobus%3Aauth%3Ascope%3Agroups.api.globus.org%3Aview_my_groups_and_memberships&state=_default&response_type=code&code_challenge=IJssgQ4lnOzvp6UBy6cEK8u_7x7lIDU-fxBkRPmU_pk&code_challenge_method=S256&access_type=online&prefill_named_grant=Login+from+deriva-client+on+0a21adc9365d+%5BLinux-6.1.58%2B-x86_64-with-glibc2.35%5D+to+hosts+%5Bdev.eye-ai.org%5D+with+requested+scopes+%5Bhttps%3A%2F%2Fauth.globus.org%2Fscopes%2Fdev.eye-ai.org%2Fderiva_all%2C+openid%2C+urn%3Aglobus%3Aauth%3Ascope%3Agroups.api.globus.org%3Av

In [3]:
EA = EyeAI(hostname = DEFAULT_SERVER, catalog_id = catalog_id )

In [4]:
# @title Initiate an Execution
configuration_records, cm = EA.execution_init(configuration_rid="2-5N5P")
configuration_records


Configuration validation successful!


2024-02-01 17:30:10,891 - INFO - Wrote configuration file: /root/.bdbag/bdbag.json
2024-02-01 17:30:10,895 - INFO - Attempting to resolve http://identifiers.org/minid:oLkKfQWmEfXw into a valid set of URLs.
2024-02-01 17:30:12,012 - INFO - The identifier minid:oLkKfQWmEfXw resolved into the following locations: [https://eye-ai-shared.s3.amazonaws.com//6656e08709b2f7c0bbed8284fdfb550d/2024-01-16_17.33.09/Dataset_2-5K2C.zip]
2024-02-01 17:30:12,021 - INFO - Attempting GET from URL: https://eye-ai-shared.s3.amazonaws.com//6656e08709b2f7c0bbed8284fdfb550d/2024-01-16_17.33.09/Dataset_2-5K2C.zip
2024-02-01 17:30:12,498 - INFO - File [/content/Dataset_2-5K2C.zip] transfer complete. 8.533 KB transferred. Elapsed time: 0:00:00.000382.
2024-02-01 17:30:12,503 - INFO - Extracting ZIP archived file: /content/Dataset_2-5K2C.zip
2024-02-01 17:30:12,510 - INFO - File /content/Dataset_2-5K2C.zip was successfully extracted to directory /content/Dataset_2-5K2C
2024-02-01 17:30:12,525 - INFO - Attempting 

{'annotation_tag_rid': '2-5N4E',
 'execution': '2-5N6E',
 'workflow': '2-5N5T',
 'process': ['2-5N5R'],
 'bag_paths': ['/content/Dataset_2-5K2C'],
 'model_paths': ['download/Merged_Cropped_Porper_or_Not_Dataset_TLBR_95_VGG19_Val_Accuracy.hdf5'],
 'configuration_path': 'download/2-5MW8_config_cropping_test.json'}

In [5]:
# @title Clone github repo
for proc in EA.configuration.process:
  !git clone https://github.com/{proc.owner}/{proc.repo}.git

Cloning into 'eye-ai-ml'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 125 (delta 30), reused 45 (delta 19), pack-reused 69
Receiving objects: 100% (125/125), 194.99 MiB | 29.43 MiB/s, done.
Resolving deltas: 100% (66/66), done.
Updating files: 100% (16/16), done.


In [6]:
# @title Data Preprocessing (Filtering Image.csv for just Field_2 Images)

# Selecting image of angle 2 (Field 2) -- Image angle vocab 2SK6;
bag_path1 = configuration_records['bag_paths'][0]
Dataset_Path = os.path.join(bag_path1, 'data/Image.csv')
Dataset = pd.read_csv(Dataset_Path)

Dataset_Field_2 = Dataset[Dataset['Image_Angle_Vocab'] == "2SK6"]

# Dataset_Field_2.head()
# for sanity check compare the filtered data length with just taking Filename column and filtering it with the same condition where it has Field 2 in it as a substring.

print(len(Dataset_Field_2))
print(len(Dataset[Dataset['Filename'].str.contains("Field 2")]))

file2_csv_path = f"{bag_path1}_Field_2.csv"
Dataset_Field_2.to_csv(file2_csv_path, index=False)

5
5


In [33]:
# @title Execute Proecss algorithm (Cropping)

os.chdir('/content/eye-ai-ml/Glaucoma Suspect or No Glaaucoma/')
from Cleaned_Optic_Disc_Cropping_Algorithm_1_0_3_SVG import preprocess_and_crop

with cm as exec:
  preprocess_and_crop(
      "/content/Dataset_2-5K2C/data/assets/Image/",
      file2_csv_path,
      '../../output/output.csv',
      'template.jpg',  # If required, otherwise remove this argument
      '../../output/',
      '/content/'+configuration_records['model_paths'][0],  # If required, otherwise remove this argument
      configuration_records['process'][0],
      configuration_records['annotation_tag_rid'],
      EA.configuration.annotation_tag.name
      )
  os.chdir('/content')

Processing image 632534_873477_10000842_Left Field 2.jpg : , trial 1, color channel grey, resize function imgResize_primary
1/1 [==============================] - 2s 2s/step
SVG for 8GER saved.
Image 632534_873477_10000842_Left Field 2.jpg (grey) cropped and saved at ../../output/Cropped_High_Resolution_8GER_632534_873477_10000842_Left Field 2_V9J0.jpg.
Processing image 632534_873477_10000843_Right Field 2.jpg : , trial 1, color channel grey, resize function imgResize_primary
1/1 [==============================] - 1s 1s/step
Processing image 632534_873477_10000843_Right Field 2.jpg : , trial 2, color channel green, resize function imgResize_primary
1/1 [==============================] - 1s 1s/step
Processing image 632534_873477_10000843_Right Field 2.jpg : , trial 3, color channel red, resize function imgResize_primary
1/1 [==============================] - 2s 2s/step
Processing image 632534_873477_10000843_Right Field 2.jpg : , trial 4, color channel blue, resize function imgResize_pr

2024-02-01 18:18:23,306 - INFO - Initializing uploader: GenericUploader v1.6.4 [Python 3.10.12, Linux-6.1.58+-x86_64-with-glibc2.35]
2024-02-01 18:18:23,307 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-02-01 18:18:23,312 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-02-01 18:18:23,315 - INFO - Checking for updated configuration...


SVG for 9YSP saved.
Image 666736_924539_10399838_Left Field 2.JPG (grey) cropped and saved at ../../output/Cropped_High_Resolution_9YSP_666736_924539_10399838_Left Field 2_V9J0.JPG.
Number of images in CSV: 5
Number of images in directory: 18
Number of images in output directory: 7
Number of cropped images: 5
--- 200.9882538318634 seconds ---
Exeption type: None, Exeption value: None, Exeption traceback: None


2024-02-01 18:18:24,224 - INFO - Updated configuration found.
2024-02-01 18:18:24,228 - INFO - Attempting to acquire a dependent lock in [/content/ExecutionAssets]
2024-02-01 18:18:24,230 - INFO - Scanning files in directory [/content/ExecutionAssets]...
2024-02-01 18:18:24,231 - INFO - File upload processing completed: 0 files were uploaded successfully, 0 files failed to upload due to errors, 0 files were skipped because they did not satisfy the matching criteria of the configuration.


In [34]:
# @title ML result analysis
import numpy as np
output_csv_path = "output/output.csv"
cropped_info = pd.read_csv(output_csv_path)[["Image RID", "Worked Image Cropping Function"]]

cropped_info['Cropped'] = np.where(cropped_info['Worked Image Cropping Function'] == 'Raw Cropped to Eye', 'False', 'True')
cropped_info = cropped_info[["Image RID", "Cropped"]]
cropped_info.rename(columns={'Image RID': 'RID'}, inplace=True)

2024-02-01 18:30:06,849 - WARNING - <ipython-input-34-0d106c82c7be>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cropped_info.rename(columns={'Image RID': 'RID'}, inplace=True)



In [35]:
# @title Upload ML results
# upload cropping bounding box
EA.upload_assets(f'/content/output/{configuration_records["process"][0]}')

# upload cropping metadata
EA.update_image_table(cropped_info)

2024-02-01 18:30:23,261 - INFO - Initializing uploader: GenericUploader v1.6.4 [Python 3.10.12, Linux-6.1.58+-x86_64-with-glibc2.35]
2024-02-01 18:30:23,265 - INFO - Creating client of type <class 'globus_sdk.services.auth.client.native_client.NativeAppAuthClient'> for service "auth"
2024-02-01 18:30:23,267 - INFO - Finished initializing AuthLoginClient. client_id='8ef15ba9-2b4a-469c-a163-7fd910c9d111', type(authorizer)=<class 'globus_sdk.authorizers.base.NullAuthorizer'>
2024-02-01 18:30:23,272 - INFO - Checking for updated configuration...
2024-02-01 18:30:24,413 - INFO - Updated configuration found.
2024-02-01 18:30:24,419 - INFO - Scanning files in directory [/content/output/2-5N5R]...
2024-02-01 18:30:24,428 - INFO - Including file: [/content/output/2-5N5R/2-5N4E_9YSC_imgResize_primary.svg].
2024-02-01 18:30:24,431 - INFO - Including file: [/content/output/2-5N5R/2-5N4E_8GER_imgResize_primary.svg].
2024-02-01 18:30:24,433 - INFO - Including file: [/content/output/2-5N5R/2-5N4E_9YS